In [2]:
!pip install spacy

In [9]:
!pip install -U spacy[transformers]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!pip install srsly

In [10]:
import json

with open("spacy_training_data.json", "r") as f:
    resume_data = json.load(f)

In [11]:
len(resume_data)

2961

In [12]:
resume_data[0]

['Ankita Nagendra Babar babarankita112@gmail.com | 8329004204   Strong proficiency in HTML5, CSS3 and JavaScript   Strong proficiency in JavaScript, including DOM manipulation and JavaScript object model   Hands-on knowledge on ReactJs, Redux   Familiarity with newer specifications of ECMAScript 6   Ability to understand business requirements and translate them into technical requirements   A knack for benchmarking and optimization   Familiarity with code versioning tools (Git)   Understanding of responsive web development with Bootstrap.   Strong verbal and written communication skills. WORK EXPERIENCE O2 Soft Solutions, Pune Nov 2019   Dec2021 Successfully developed and maintained School ERP. Was responsible for developing modules, maintaining and testing them. EDUCATION University of Pune, Pune.   M.E. (Computer Engineering) 2016 First Class(7.5 SGPA) from TSSM s Bhivarabai Sawant College Of Engineering and research, Narhe, Pune   B.E. (Computer Engineering) 2014 Higher Second Class

In [13]:
!python -m spacy init fill-config /content/base_config.cfg ./config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [20]:
import spacy
import random
from spacy.tokens import DocBin
from spacy.training import Example
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [21]:
unique_data = {}
for text, ann in resume_data:
    key = text.strip().lower()
    if key not in unique_data:
        unique_data[key] = (text, ann)

deduped_data = list(unique_data.values())
random.seed(42)
random.shuffle(deduped_data)

In [25]:
len(deduped_data)

2701

In [22]:
train_data, dev_data = train_test_split(deduped_data, test_size=0.2, random_state=42)

In [23]:
len(train_data)

2160

In [24]:
len(dev_data)

541

In [26]:
train_texts = {text.strip().lower() for text, _ in train_data}
dev_texts = {text.strip().lower() for text, _ in dev_data}
overlap = train_texts.intersection(dev_texts)

print(f"Overlapping texts: {len(overlap)}")

Overlapping texts: 0


In [27]:
def save_data_to_spacy_file(data, nlp, output_path, log_path = None):
    doc_bin = DocBin()
    with open(log_path, "w") if log_path else open("/dev/null", "w") as log_file:
        for text, annot in tqdm(data):
            doc = nlp.make_doc(text)
            ents = []
            entity_indices = set()

            for start, end, label in annot["entities"]:
                if any(i in entity_indices for i in range(start, end)):
                    continue
                span = doc.char_span(start, end, label = label, alignment_mode = "contract")
                if span is None:
                    log_file.write(f"Skipping entity [{label}] at {start}-{end}: {text[start:end]}\n")
                    continue
                ents.append(span)
                entity_indices.update(range(start, end))

            doc.ents = ents
            doc_bin.add(doc)

    doc_bin.to_disk(output_path)

nlp = spacy.blank("en")

save_data_to_spacy_file(train_data, nlp, "train.spacy", log_path = "skipped_entities.txt")
save_data_to_spacy_file(dev_data, nlp, "dev.spacy", log_path = "skipped_entities.txt")
print("train.spacy and dev.spacy created.")

100%|██████████| 541/541 [00:03<00:00, 167.00it/s]


train.spacy and dev.spacy created.


In [28]:
!python -m spacy train config.cfg --output ./output --paths.train /content/train.spacy --paths.dev /content/dev.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 194kB/s]
config.json: 100% 481/481 [00:00<00:00, 4.17MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 6.22MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 3.14MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 6.65MB/s]
2025-05-04 17:57:53.739597: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746381473.959619   16156 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746381474.015889   16156 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2

In [29]:
!python -m spacy evaluate --gpu-id 0 output/model-best dev.spacy

ℹ Using GPU: 0
2025-05-04 19:12:01.677992: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746385921.705846   34450 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746385921.714145   34450 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-04 19:12:01.746385: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Token indices sequence length is longer than the specified maximum sequence length for this model (542

In [31]:
import spacy

nlp = spacy.load("output/model-best")
doc = nlp("John Doe is a React developer from Pune with 5 years of experience.")

for ent in doc.ents:
    print(ent.text, ent.label_)


Pune LOC


In [32]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.7 MB/s eta 0:00:00


In [33]:
import spacy
import re
from PyPDF2 import PdfReader

nlp = spacy.load("output/model-best")

def clean_resume(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.encode('ascii', errors='ignore').decode()
    text = text.strip()
    return text

pdf_path = "/content/Arpita_Python Developer_Atmecs_4yrs_Pune.pdf"

reader = PdfReader(pdf_path)
extracted_text = ''
for page in reader.pages:
    if page.extract_text():
        extracted_text += page.extract_text()

cleaned_resume = clean_resume(extracted_text)

doc = nlp(cleaned_resume)

print("Entities found in Resume:")
print("="*30)
for ent in doc.ents:
    print(f"{ent.text} --> {ent.label_}")


Entities found in Resume:
Arpita Ghansham Badwaik --> PERSON
+91 8483003504 --> PHONE
arpita.badwaik4@gmail.com --> EMAIL
Python --> SKILL
Python 2 --> SKILL
Python 3 --> SKILL
Django --> SKILL
Django 1.8 --> SKILL
Django 3.0 --> SKILL
GCP --> SKILL
Cloud functions --> SKILL
App Engine --> SKILL
Firestore --> SKILL
Cloud Scheduler --> SKILL
PubSub --> SKILL
Drive APIs --> SKILL
AI and ML --> SKILL
Pillow --> SKILL
Numpy --> SKILL
RDMS --> SKILL
MySQL --> SKILL
Postgres --> SKILL
HTML --> SKILL
Bootstrap --> SKILL
CSS --> SKILL
Tkinter --> SKILL
JavaScript --> SKILL
jQuery --> SKILL
Ajax --> SKILL
Git --> SKILL
Rest API --> SKILL
Asynchronous process --> SKILL
State machines --> SKILL
Signals --> SKILL
Celery --> SKILL
Cron tab --> SKILL
Decorators --> SKILL
Digital Ocean --> SKILL
Nginx --> SKILL
Firestore --> SKILL
Google drive APIs --> SKILL
Samunnati --> PROJECT_TITLE
Nodejs --> SKILL
Image To Sketch --> PROJECT_TITLE
Dassault Systems (Solidworks) --> PROJECT_ORG
Python --> SKILL
Pi

In [34]:
import spacy
import re
from PyPDF2 import PdfReader

nlp = spacy.load("output/model-best")

def clean_resume(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.encode('ascii', errors='ignore').decode()
    text = text.strip()
    return text

pdf_path = "/content/Naukri_PradipShankarParkale[5y_10m].pdf"

reader = PdfReader(pdf_path)
extracted_text = ''
for page in reader.pages:
    if page.extract_text():
        extracted_text += page.extract_text()

cleaned_resume = clean_resume(extracted_text)

doc = nlp(cleaned_resume)

print("Entities found in Resume:")
print("="*30)
for ent in doc.ents:
    print(f"{ent.text} --> {ent.label_}")

Entities found in Resume:
Pradip Parkale --> PERSON
8208052167/8237391043 --> PHONE
pradipparkale269@gmail.com --> EMAIL
Pune --> LOC
Python --> SKILL
Javascript --> SKILL
ReactJS --> SKILL
Flask --> SKILL
Performance Testing --> SKILL
Jenkins --> SKILL
PostgreSQL --> SKILL
Git --> SKILL
SQLLite --> SKILL
Jira --> SKILL
SQLAlchemy --> SKILL
Material UI --> SKILL
Sep '17 - Present --> PROJECT_DURATION
EDB Pune Pune --> PROJECT_ORG
Oct '16 - Sep --> DATE
Sedemac Pune Pune, MH --> ORG
B. E. in Electronics and Tele- Comminication Jun --> DEGREE
Pune university Pune, MH --> ORG
VIIT Pune Diploma. in Electronics and Tele- Comminication Jun --> ORG
MSBTC, Maharashtra --> ORG
Goverment Polytechnic, Washim --> ORG
JavaScript --> SKILL
Python --> SKILL
JQuery --> SKILL
CSS --> SKILL
HTML --> SKILL
React.js --> SKILL
Webpack --> SKILL
Flask --> SKILL
React Hooks --> SKILL
Babel --> SKILL
Jasmine --> SKILL
SqlliteDB --> SKILL
Bootstrap --> SKILL
Flask --> SKILL
-table --> SKILL
react-virtulization

In [36]:
import shutil

model_path = "output/model-best"

zip_path = "model-best.zip"

shutil.make_archive("model-best", 'zip', model_path)

'/content/model-best.zip'

In [37]:
from google.colab import files
files.download("model-best.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>